In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
from tqdm.auto import tqdm
import concurrent.futures
from multiprocessing import Pool
import copy,os,sys
from collections import Counter,deque
import matplotlib.pyplot as plt
import json
import functools
import itertools
import math

In [3]:
import numpy as np
import pandas as pd

$$ f_d=max(0,min(1,1-\gamma*ln(\beta*x)))$$
$$ $$
$$ f_d=1; x \in [0,\frac{1}{\beta}] $$
$$ $$
$$ f_d=0; x=\frac{e^{\frac{1}{\gamma}}}{\beta} $$

# Netowrkx

In [55]:
import networkx as nx

In [ ]:
G = nx.Graph()
G.add_edge('A', 'B', weight=4)
G.add_edge('B', 'D', weight=2)
G.add_edge('A', 'C', weight=3)
G.add_edge('C', 'D', weight=4)
nx.shortest_path(G, 'A', 'D', weight='weight')
['A', 'B', 'D']

In [77]:
driver={
    "A":np.array([3,0,4]),
    "B":np.array([2,1,3]),
    "C":np.array([0,0,5])
}

relations=np.vstack(list(driver.values()))
relations

array([[3, 0, 4],
       [2, 1, 3],
       [0, 0, 5]])

In [163]:
class KMAlgo:
    def __init__(self, N, M, matrix):
        self.N = N
        self.M = M
        # 声明二分图
        self.adj_matrix = matrix  # np array with dimension N*M

        # 初始化顶标
        self.label_left = np.max(self.adj_matrix, axis=1)  # init label for the left set
        self.label_right = np.zeros(self.M)  # init label for the right set

        # 初始化匹配结果
        self.match_right = np.empty(self.N, dtype=int) * np.nan

        # 初始化辅助变量
        self.visit_left = np.zeros(self.N, dtype=bool)
        self.visit_right = np.zeros(self.M, dtype=bool)
        self.slack_right = np.ones(self.M, dtype=int) * np.inf

    # 寻找增广路，深度优先
    def find_path(self, i):
        self.visit_left[i] = True
        for j, match_weight in enumerate(self.adj_matrix[i]):
            # 已被匹配（解决递归中的冲突）
            if self.visit_right[j]:
                continue
            gap = self.label_left[i] + self.label_right[j] - match_weight
            if gap == 0:
                # 找到可行匹配
                self.visit_right[j] = True
                # j未被匹配，或虽然j已被匹配，但是j的已匹配对象有其他可选备胎
                if np.isnan(self.match_right[j]) or self.find_path(int(self.match_right[j])):
                    self.match_right[j] = i
                    return True
            else:
                # 计算变为可行匹配需要的顶标改变量
                self.slack_right[j] = min(self.slack_right[j], gap)
                if np.isinf(self.slack_right[j]):
                    self.slack_right[j] = gap
        return False

    # KM主函数
    def solve(self):
        for i in range(self.N):
            # 重置辅助变量
            self.slack_right = np.ones(self.N) * np.inf
            print("---- looping at i: %s ----" % i)
            while True:
                # 重置辅助变量
                self.visit_left = np.zeros(self.N, dtype=bool)
                self.visit_right = np.zeros(self.M, dtype=bool)
                print("[init]")
                self.dlog(i)
                # 能找到可行匹配
                if self.find_path(i):
                    print("\n[found match]")
                    self.dlog(i)
                    break
                # 不能找到可行匹配，修改顶标
                # (1)将所有在增广路中的X方点的label全部减去一个常数d
                # (2)将所有在增广路中的Y方点的label全部加上一个常数d
                print("\n[not-found match | calc d]")
                self.dlog(i)
                d = np.inf
                for j, slack in enumerate(self.slack_right):
                    if not self.visit_right[j] and slack < d:
                        d = slack
                for k in range(self.N):
                    if self.visit_left[k]:
                        self.label_left[k] -= d
                for n in range(self.M):
                    if self.visit_right[n]:
                        self.label_right[n] += d
                    else:
                        print("why for : %s" % n)
                        self.slack_right[n] -= d
                print("\n[not-found match | minus d=%s]" % d)
                if np.all(self.slack_right==[4.,4.,0.]):
                    break
                self.dlog(i)
            if i == 2:
                break

        match_res = 0
        for j in range(self.N):
            if self.M > self.match_right[j] >= 0:
                match_res += self.adj_matrix[int(self.match_right[j])][j]
        return match_res
    
    def dlog(self,i=-1):
        print("at %s" % i)
        print(">>> match:",self.match_right)
        print(">>> label_left:",self.label_left)
        print(">>> label_right:",self.label_right)
        print(">>> visit_left:",self.visit_left)
        print(">>> visit_right:",self.visit_right)
        print(">>> slack_right:",self.slack_right)



In [164]:
km = KMAlgo(relations.shape[0],relations.shape[1],relations)
km.dlog()
print(">>> adj_matrix")
print(km.adj_matrix)
km.solve()
print(">>> match result:", km.match_right)

at -1
>>> match: [nan nan nan]
>>> label_left: [4 3 5]
>>> label_right: [0. 0. 0.]
>>> visit_left: [False False False]
>>> visit_right: [False False False]
>>> slack_right: [inf inf inf]
>>> adj_matrix
[[3 0 4]
 [2 1 3]
 [0 0 5]]
---- looping at i: 0 ----
[init]
at 0
>>> match: [nan nan nan]
>>> label_left: [4 3 5]
>>> label_right: [0. 0. 0.]
>>> visit_left: [False False False]
>>> visit_right: [False False False]
>>> slack_right: [inf inf inf]

[found match]
at 0
>>> match: [nan nan  0.]
>>> label_left: [4 3 5]
>>> label_right: [0. 0. 0.]
>>> visit_left: [ True False False]
>>> visit_right: [False False  True]
>>> slack_right: [ 1.  4. inf]
---- looping at i: 1 ----
[init]
at 1
>>> match: [nan nan  0.]
>>> label_left: [4 3 5]
>>> label_right: [0. 0. 0.]
>>> visit_left: [False False False]
>>> visit_right: [False False False]
>>> slack_right: [inf inf inf]

[not-found match | calc d]
at 1
>>> match: [nan nan  0.]
>>> label_left: [4 3 5]
>>> label_right: [0. 0. 0.]
>>> visit_left: [ Tru

6

>>> match result: [ 1. nan  0.]


In [71]:
driver={
    "A":np.array([3,0,4]),
    "B":np.array([2,1,3]),
    "C":np.array([0,0,5])
}

relation=np.vstack(list(driver.values()))
match=np.zeros_like(relation)
driver_score = np.max(relation, axis=1)
leader_score = np.zeros(relation.shape[1])
driver_vis = np.zeros(relation.shape[0])
leader_vis = np.zeros(relation.shape[1])



print(">>> relation:")
print(relation)
print(">>> match:")
print(match)
print(">>> driver_score:")
print(driver_score)
print(">>> leader_socre:")
print(leader_score)

def find():
    pass

for rowIdx,row in enumerate(relation):
    while(True):
        driver_vis.fill(0)
        leader_vis.fill(0)
        colIdx = np.argmax(row)
        # 不能是已经匹配过的点(colIdx)
        if not np.any(match[:,colIdx]):
            # 还要满足两边顶点score和等于边权重(相等子图)
            if relation[rowIdx,colIdx] == driver_score[rowIdx] + leader_score[colIdx]:
                match[rowIdx,colIdx] = 1
                break
        else:
            # 1. 匹配冲突时，考虑降低效率，取次大匹配
            # 1.1 计算“新增匹配”的次大匹配及带来的降效
            newMatch_w = relation[rowIdx,:]
            newMatch_2ndMax = np.max(np.delete(newMatch_w,colIdx))
            newMatch_wReduce = newMatch_w[colIdx] - newMatch_2ndMax
            # 1.2 计算“新增匹配”的次大匹配及带来的降效
            oldMatchIdx = match[:,colIdx].nonzero()[0][0]
            oldMatch_w = relation[oldMatchIdx,:]
            oldMatch_2ndMax = np.max(np.delete(oldMatch_w,colIdx))
            oldMatch_wReduce = oldMatch_w[colIdx] - oldMatch_2ndMax
    #         print(newMatch_w,newMatch_2ndMax,newMatch_wReduce)
    #         print(oldMatch_w,oldMatch_2ndMax,oldMatch_wReduce)
            # 选择降效更低的方案 | 相同时随便选哪个都可以（wReduce==newMatch_wReduce==oldMatch_wReduce，最后选的时候直接取max)
            wReduce = newMatch_wReduce if newMatch_wReduce <= oldMatch_wReduce else oldMatch_wReduce
            driver_score[rowIdx] -= wReduce
            driver_score[oldMatchIdx] -= wReduce
            leader_score[colIdx] += wReduce
            # 2. 访问过的全都重新匹配
    #         break
            pass
    print("\n"*2+">>> after step: {}".format(rowIdx))
    print(">>> relation:")
    print(relation)
    print(">>> match:")
    print(match)
    print(">>> driver_score:")
    print(driver_score)
    print(">>> leader_socre:")
    print(leader_score)

>>> relation:
[[3 0 4]
 [2 1 3]
 [0 0 5]]
>>> match:
[[0 0 0]
 [0 0 0]
 [0 0 0]]
>>> driver_score:
[4 3 5]
>>> leader_socre:
[0. 0. 0.]


>>> after step: 0
>>> relation:
[[3 0 4]
 [2 1 3]
 [0 0 5]]
>>> match:
[[0 0 1]
 [0 0 0]
 [0 0 0]]
>>> driver_score:
[4 3 5]
>>> leader_socre:
[0. 0. 0.]


KeyboardInterrupt: 

In [ ]:
class max_bipartite_graph_match(object):
 
    def __init__(self,graph):
        self.graph = graph
        self.max_weight = graph.sum()
        self.n,self.m = graph.shape
        assert self.n == self.m
        self.lx = self.graph.max(1)
        self.ly = np.array([0] * self.m, dtype=int) #if weight of edges is float, change dtype to float
        self.match = np.array([-1] * self.n, dtype=int)
        self.slack = np.array([0] * self.m, dtype=int)
        self.visx = np.array([False] * self.n, dtype=bool)
        self.visy = np.array([False] * self.m, dtype=bool)
 
    def reset_slack(self):
        self.slack.fill(self.max_weight + 1)
 
    def reset_vis(self):
        self.visx.fill(False)
        self.visy.fill(False)
 
    def find_path(self, x):
        self.visx[x] = True
 
        for y in range(self.m):
            if self.visy[y]: continue
            tmp_delta = self.lx[x] + self.ly[y] - self.graph[x][y]
            if  tmp_delta == 0:
                self.visy[y] = True
                if  self.match[y] == -1 or self.find_path(self.match[y]):
                    self.match[y] = x
                    return True
            elif self.slack[y] > tmp_delta:
                self.slack[y] = tmp_delta
 
        return False
 
    def KM(self):
        for x in range(self.n):
            self.reset_slack()
            while True:
                self.reset_vis()
                if self.find_path(x): break
                else: #update slack
                    delta = self.slack[~self.visy].min()
                    self.lx[self.visx] -= delta
                    self.ly[self.visy] += delta
                    self.slack[~self.visy] -= delta
 
        return np.sum(self.lx) + np.sum(self.ly)
 
    def __call__(self):


In [27]:
BASE_DIR="/Users/didi/Downloads/0-排车相关/"
detail_fp = os.path.join(BASE_DIR,"0205_detail")
driver2leader_fp = os.path.join(BASE_DIR,"transInfo_grouped.csv")

In [3]:
class Driver:
    def __init__(self,value,links,line):
        self.value = value
        self.links = links
        self.line = line
    

class Link:
    def __init__(self,w,to):
        self.w = w
        self.to = to


class Line:
    def __init__(self,value,leaders,driver):
        self.value = value
        self.leaders = leaders
        self.driver = driver
        

In [93]:
d2lDF = pd.read_csv(driver2leader_fp,sep="\t")[["driver_phone","leader_uid","dateSumNormed","vl_volume","vl_capacity"]]
detailDF = pd.read_csv(detail_fp,sep="\t").rename(columns={"tuan_ids":"leader_uid"})
detailDF["leader_uid"] = detailDF["leader_uid"].apply(lambda x:[i for i in x.split(",") if len(i)>1])
detailDF = detailDF.explode("leader_uid")
detailDF["leader_uid"]=detailDF["leader_uid"].astype(np.int64)

In [94]:
jdf = detailDF.merge(d2lDF,on=["driver_phone","leader_uid"],how="left")
jdf
jdf["dateSumNormed"].nunique()
jdf["dateSumNormed"].nunique(False)

,line_id,volume,weight,driver_phone,leader_uid,dateSumNormed,vl_volume,vl_capacity
0,1328,1081.393,385.201,2b55b0efff57f4c1920048f3ea9deada,639264856631565102,74950.0,4800.0,1100.0
1,1328,1081.393,385.201,2b55b0efff57f4c1920048f3ea9deada,639264856638408585,88308.0,4800.0,1100.0
2,1328,1081.393,385.201,2b55b0efff57f4c1920048f3ea9deada,637312124015608801,81296.0,4800.0,1100.0
3,1328,1081.393,385.201,2b55b0efff57f4c1920048f3ea9deada,637312124013150198,81296.0,4800.0,1100.0
4,1328,1081.393,385.201,2b55b0efff57f4c1920048f3ea9deada,639264856660669332,73571.0,4800.0,1100.0
...,...,...,...,...,...,...,...,...
4616,4,2230.947,728.780,ee00b34ea5f0737fbf2dec002f0304f4,639264856637531694,92988.0,8500.0,1400.0
4617,8,908.631,264.807,f7a4370d338593d405dd5223dd5f27c9,637276942179510865,NaN,NaN,NaN
4618,8,908.631,264.807,f7a4370d338593d405dd5223dd5f27c9,639264856678287441,NaN,NaN,NaN
4619,10,428.246,117.941,463475e14077c5a094b9bbfbf4ba7db5,637312124012127560,NaN,NaN,NaN


563

564

In [51]:
len(set(d2lDF["driver_phone"]))
detailDF.shape


478

(259, 5)

In [52]:
detailDF.merge(d2lDF,on=["driver_phone","leader_uid"],how="left")


ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [ ]:
relation = []

In [ ]:
def load_file(fp):
    with open(fp,"r") as fr:
        for i in fr:
            yield i



gtr = load_file(detail_fp)
print("\n".join(itertools.islice(gtr,2)))
print("\n".join(itertools.islice(gtr,2)))